In [1]:
import ctypes
import math
import random

In [2]:
import os
import subprocess

In [3]:
def c_array(ctype, values):
    arr = (ctype*len(values))()
    arr[:] = values
    return arr

class BOX(ctypes.Structure):
    _fields_ = [("x", ctypes.c_float),
                ("y", ctypes.c_float),
                ("w", ctypes.c_float),
                ("h", ctypes.c_float)]

class DETECTION(ctypes.Structure):
    _fields_ = [("bbox", BOX),
                ("classes", ctypes.c_int),
                ("prob", ctypes.POINTER(ctypes.c_float)),
                ("mask", ctypes.POINTER(ctypes.c_float)),
                ("objectness", ctypes.c_float),
                ("sort_class", ctypes.c_int)]


class IMAGE(ctypes.Structure):
    _fields_ = [("w", ctypes.c_int),
                ("h", ctypes.c_int),
                ("c", ctypes.c_int),
                ("data", ctypes.POINTER(ctypes.c_float))]

class METADATA(ctypes.Structure):
    _fields_ = [("classes", ctypes.c_int),
                ("names", ctypes.POINTER(ctypes.c_char_p))]

In [8]:
from cached_property import cached_property

In [12]:
class Darknet(object):
    def __init__(self, darknet_directory=".."):
        self.darknet_directory=os.path.abspath(darknet_directory)
        
    @property
    def cfg_dir(self):
        return os.path.abspath(
            os.path.join(
                self.darknet_directory,
                "cfg"
            )
        )
    
    @property
    def exe(self):
        return os.path.abspath(
            os.path.join(
                self.darknet_directory,
                "darknet"
            )
        )
    
    @property
    def lib_path(self):
        return os.path.abspath(
            os.path.join(
                self.darknet_directory,
                "libdarknet.so"
            )
        )
    
    def build(self, force=False, **kwargs):
        if os.path.exists(self.exe) and not force:
            return ""
        os.chdir(self.darknet_directory)
        out = subprocess.check_output(["make", "clean"])
       
        for arg, value in kwargs.items():
            if value:     
                os.environ[arg.upper()]="1"
                #print("{}=1".format(arg.upper()))
            else:
                os.environ[arg.upper()]="0"
                #print("{}=0".format(arg.upper()))
                
        out = subprocess.check_output(["make", "-j8"])
        return (out.decode("UTF-8"))
    
    @cached_property
    def lib(self):
        return ctypes.CDLL(self.lib_path, ctypes.RTLD_GLOBAL)
        


        
dn = Darknet("/projects/darknet/")
out = dn.build(opencv=True, openmp=True)

In [13]:
dn.lib

<CDLL '/projects/darknet/libdarknet.so', handle 562eaa068830 at 0x7f37677b0b00>

In [ ]:
# Test
"""
for opencv in [True, False]:
    for openmp in [True, False]:
        out = dn.build(opencv=opencv, openmp=openmp)
        out=out.lower()
        assert ("opencv" in out) == opencv
        assert ("openmp" in out) == openmp
        print()"""

In [ ]:
def sample(probs):
    s = sum(probs)
    probs = [a/s for a in probs]
    r = random.uniform(0, 1)
    for i in range(len(probs)):
        r = r - probs[i]
        if r <= 0:
            return i
    return len(probs)-1



In [ ]:
#lib = CDLL("/home/pjreddie/documents/darknet/libdarknet.so", RTLD_GLOBAL)

lib.network_width.argtypes = [c_void_p]
lib.network_width.restype = c_int
lib.network_height.argtypes = [c_void_p]
lib.network_height.restype = c_int

In [ ]:
predict = lib.network_predict
predict.argtypes = [c_void_p, POINTER(c_float)]
predict.restype = POINTER(c_float)

set_gpu = lib.cuda_set_device
set_gpu.argtypes = [c_int]

make_image = lib.make_image
make_image.argtypes = [c_int, c_int, c_int]
make_image.restype = IMAGE

get_network_boxes = lib.get_network_boxes
get_network_boxes.argtypes = [c_void_p, c_int, c_int, c_float, c_float, POINTER(c_int), c_int, POINTER(c_int)]
get_network_boxes.restype = POINTER(DETECTION)

make_network_boxes = lib.make_network_boxes
make_network_boxes.argtypes = [c_void_p]
make_network_boxes.restype = POINTER(DETECTION)

free_detections = lib.free_detections
free_detections.argtypes = [POINTER(DETECTION), c_int]

free_ptrs = lib.free_ptrs
free_ptrs.argtypes = [POINTER(c_void_p), c_int]

network_predict = lib.network_predict
network_predict.argtypes = [c_void_p, POINTER(c_float)]

reset_rnn = lib.reset_rnn
reset_rnn.argtypes = [c_void_p]

load_net = lib.load_network
load_net.argtypes = [c_char_p, c_char_p, c_int]
load_net.restype = c_void_p

do_nms_obj = lib.do_nms_obj
do_nms_obj.argtypes = [POINTER(DETECTION), c_int, c_int, c_float]

do_nms_sort = lib.do_nms_sort
do_nms_sort.argtypes = [POINTER(DETECTION), c_int, c_int, c_float]

free_image = lib.free_image
free_image.argtypes = [IMAGE]

letterbox_image = lib.letterbox_image
letterbox_image.argtypes = [IMAGE, c_int, c_int]
letterbox_image.restype = IMAGE

load_meta = lib.get_metadata
lib.get_metadata.argtypes = [c_char_p]
lib.get_metadata.restype = METADATA

load_image = lib.load_image_color
load_image.argtypes = [c_char_p, c_int, c_int]
load_image.restype = IMAGE

rgbgr_image = lib.rgbgr_image
rgbgr_image.argtypes = [IMAGE]

predict_image = lib.network_predict_image
predict_image.argtypes = [c_void_p, IMAGE]
predict_image.restype = POINTER(c_float)

def classify(net, meta, im):
    out = predict_image(net, im)
    res = []
    for i in range(meta.classes):
        res.append((meta.names[i], out[i]))
    res = sorted(res, key=lambda x: -x[1])
    return res

def detect(net, meta, image, thresh=.5, hier_thresh=.5, nms=.45):
    im = load_image(image, 0, 0)
    num = c_int(0)
    pnum = pointer(num)
    predict_image(net, im)
    dets = get_network_boxes(net, im.w, im.h, thresh, hier_thresh, None, 0, pnum)
    num = pnum[0]
    if (nms): do_nms_obj(dets, num, meta.classes, nms);

    res = []
    for j in range(num):
        for i in range(meta.classes):
            if dets[j].prob[i] > 0:
                b = dets[j].bbox
                res.append((meta.names[i], dets[j].prob[i], (b.x, b.y, b.w, b.h)))
    res = sorted(res, key=lambda x: -x[1])
    free_image(im)
    free_detections(dets, num)
    return res

In [ ]:
import os

In [ ]:
net = load_net(os.path.abspath("../cfg/yolov3.cfg").encode(), os.path.abspath("../cfg/yolov3.weights").encode(), 0)

In [ ]:
dog_jpg = os.path.abspath("../data/dog.jpg")
assert os.path.exists(dog_jpg)

In [ ]:
dog = load_image(dog_jpg.encode(), 0, 0)

In [ ]:
num = c_int(0)
pnum = pointer(num)
predict_image(net, dog)

In [ ]:
meta = load_meta(os.path.abspath("../cfg/coco.data").encode())

In [ ]:
if __name__ == "__main__":
    #net = load_net("cfg/densenet201.cfg", "/home/pjreddie/trained/densenet201.weights", 0)
    #im = load_image("data/wolf.jpg", 0, 0)
    #meta = load_meta("cfg/imagenet1k.data")
    #r = classify(net, meta, im)
    #print r[:10]
    net = load_net("cfg/tiny-yolo.cfg", "tiny-yolo.weights", 0)
    meta = load_meta("cfg/coco.data")
    r = detect(net, meta, "data/dog.jpg")
    print r
    